In [1]:
%cd ../..

/home/shapkin/effective-inference


## Import libs

In [11]:
import yaml
import os
import h5py
import numpy as np
import torch

from utils.prepare_dataset import load_datasets, cut_datasets
from utils.dataset_cache import cache_embeddings, get_dataset_for_regression, build_dataset_from_cached, load_cached_dataset
from utils.config import ConfigWrapper
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModel
from numpy.random import shuffle

## Project configuration

In [3]:
config_path = 'config.yaml'

with open(config_path, "r") as f:
    config = ConfigWrapper(yaml.load(f, Loader=yaml.FullLoader))

In [4]:
if not os.path.exists(config.data.data_path):
    os.makedirs(config.data.data_path)
else:
    print('This directory already exists')

In [5]:
train_datasets = load_datasets(config.data.train_datasets, config.data.cut_size)

In [6]:
train_datasets

{'imdb': DatasetDict({
     train: Dataset({
         features: ['text', 'label'],
         num_rows: 25000
     })
     test: Dataset({
         features: ['text', 'label'],
         num_rows: 25000
     })
     unsupervised: Dataset({
         features: ['text', 'label'],
         num_rows: 50000
     })
 })}

In [7]:
tokenizer = AutoTokenizer.from_pretrained(config.model.model_name, max_length=config.general.max_len)
initial_model = AutoModel.from_pretrained(config.model.model_name).to(config.general.device)

## Cache embeddings

In [8]:
cache_embeddings(config, train_datasets, tokenizer, initial_model)

0it [00:00, ?it/s]

## Cache dataset

In [9]:
X_train, y_train, X_test, y_test = get_dataset_for_regression(config, train_datasets, tokenizer, initial_model, layer=0, heads=[0, 1])

  0%|          | 0/25000 [00:00<?, ?it/s]

In [10]:
X_train.shape, X_test.shape

((3213, 1539), (169, 1539))

## Get X_train from cached

In [11]:
X_train, y_train, X_test, y_test = build_dataset_from_cached(config, train_datasets, layer=0, heads=[0, 1])


KeyboardInterrupt



In [ ]:
X_train.shape, X_test.shape

In [13]:
X_train, y_train, X_test, y_test = load_cached_dataset(config)

In [14]:
X_train.shape, X_test.shape

((1862, 1538), (98, 1538))